<a href="https://colab.research.google.com/github/Ijeoma-Charity/ai-ml-experiments/blob/main/Input_Target_pairs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
from google.colab import files
uploaded = files.upload()

Saving the_verdict.docx to the_verdict (1).docx


In [24]:
pip install python-docx pandas

In [25]:
from docx import Document

In [26]:
doc = Document("the_verdict.docx")
full_text = '/n'.join(p.text for p in doc.paragraphs)

In [27]:
with open("the_verdict.txt", "w", encoding = "utf-8") as f:
  f.write(full_text)

In [28]:
with open("the_verdict.txt", "r", encoding = "utf-8") as f:
  raw_text = f.read()

installing tiktoken

In [29]:
! pip3 install tiktoken

In [30]:
import tiktoken

Initializing the tokenizer

In [31]:
tokenizer = tiktoken.get_encoding("gpt2")

In [32]:
enc_text = tokenizer.encode(raw_text)
enc_sample = enc_text[50:]
print(len(enc_sample))

5264


Introduction into the input-target pair concept

In [33]:
#Understanding the basics of the input-target pair
context_size = 4
x = enc_sample[:context_size]
y = enc_sample[1:context_size + 1]

print(f"x: {x}")
print(f"y:      {y}")

x: [326, 11, 287, 262]
y:      [11, 287, 262, 6001]


In [34]:
for i in range(1, context_size+1):
  context = enc_sample[:i]
  desired = enc_sample[i]

print(context, "---->", desired)
print(tokenizer.decode(context), "---->", tokenizer.decode([desired]))

[326, 11, 287, 262] ----> 6001
 that, in the ---->  height


Creating a Data Loader

In [47]:
from torch.utils.data import Dataset, DataLoader

class GPTDatasetV1(Dataset):
  def __init__(self, text, tokenizer, max_length, stride):
    self.input_ids = []
    self.target_ids = []

  #tokenize the entire text
    token_ids = tokenizer.encode(text,allowed_special = {"<|endoftext|>"})

  #implementing the sliding window to chunk the book into overlapping sequence of max_length
    for i in range(0, len(token_ids)-max_length, stride):
      input_chunk = token_ids[i:i+max_length]
      target_chunk = token_ids[i+1:i+max_length+1]
      self.input_ids.append(torch.tensor(input_chunk))
      self.target_ids.append(torch.tensor(target_chunk))

  def __len__(self):
    return len(self.input_ids)

  def __getitem__(self, idx):
    return self.input_ids[idx],self.target_ids[idx]

Creating a data loader

In [49]:
def create_dataloader_v1(text,batch_size=4,max_length=256,stride=128,shuffle=True,drop_last=True,num_workers=0):

  #initialize tokenizer
  tokenizer = tiktoken.get_encoding("gpt2")

  #creating a dataset
  dataset = GPTDatasetV1(text, tokenizer, max_length,stride)

  #creating a dataloader
  dataloader = DataLoader(
      dataset,
      batch_size=batch_size,
      shuffle=shuffle,
      drop_last=drop_last,
      num_workers=num_workers
  )

  return dataloader

Testing the data loader

In [37]:
with open("the_verdict.txt", "r", encoding="utf-8") as f:
  raw_text = f.read()

In [38]:
import torch
print("Pytorch version: ", torch.__version__)

Pytorch version:  2.9.0+cpu


In [50]:
dataloader = create_dataloader_v1(raw_text,batch_size=1,stride=1,max_length=4,shuffle=False)

In [51]:
data_iter = iter(dataloader)
first_batch = next(data_iter)
print(first_batch)

[tensor([[  464,  4643, 11600,  1717]]), tensor([[ 4643, 11600,  1717,   342]])]


In [52]:
second_batch = next(data_iter)
print(second_batch)

[tensor([[ 4643, 11600,  1717,   342]]), tensor([[11600,  1717,   342,   854]])]


Experimenting the effect of batch_size

In [57]:
dataloader = create_dataloader_v1(raw_text,batch_size=8,max_length=4,stride=4,shuffle=False)

In [58]:
data_iter = iter(dataloader)
inputs, targets = next(data_iter)
print("inputs:\n", inputs)
print("\nTargets:\n", targets)

inputs:
 tensor([[  464,  4643, 11600,  1717],
        [  342,   854, 41328, 40417],
        [ 1475,  9213,   422, 11145],
        [  271,  1668,   319,  3426],
        [ 1596,    11, 32190,   352],
        [  314,   367,  2885,  1464],
        [ 1807,  3619,   402,   271],
        [10899,  2138,   257,  7026]])

Targets:
 tensor([[ 4643, 11600,  1717,   342],
        [  854, 41328, 40417,  1475],
        [ 9213,   422, 11145,   271],
        [ 1668,   319,  3426,  1596],
        [   11, 32190,   352,   314],
        [  367,  2885,  1464,  1807],
        [ 3619,   402,   271, 10899],
        [ 2138,   257,  7026, 15632]])
